# Convolutional Autoencoder with K-Means for MNIST

In [ ]:
import torch
import numpy as np
from torchvision import datasets
import torchvision
from torch.utils.data.sampler import SubsetRandomSampler
from torch.utils.tensorboard import SummaryWriter
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F
import time

# to import MNIST as torch tensor
transform = transforms.ToTensor()

# load the training and test datasets
train_data = datasets.MNIST(root='data', train=True,
                                   download=True, transform=transform)
test_data = datasets.MNIST(root='data', train=False,
                                  download=True, transform=transform)

## training and testing dataloaders

In [ ]:
num_workers = 0
# how many samples per batch to load
batch_size = 100
lr = 1e-3
# how many epochs for training
num_epochs = 25
# latent space size
z_dim = 10

# prepare data loaders
train_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size, num_workers=num_workers)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=batch_size, num_workers=num_workers)

## autoencoder neural network design

In [ ]:
class cae(nn.Module):
    def __init__(self):
        super(cae, self).__init__()
        # convolutional encoder
        self.encoder = nn.Sequential(
            nn.Conv2d(1, 16, 3, stride=3, padding=1),  # 16, 10, 10
            nn.ReLU(True),
            nn.MaxPool2d(2, stride=2),  # 16, 5, 5
            nn.Conv2d(16, 32, 3, stride=2, padding=1),  # 32, 3, 3
            nn.ReLU(True),
            nn.MaxPool2d(2, stride=1)  # 32, 2, 2
        )
        # autoencoder bottle neck
        self.latent = nn.Sequential(
            nn.Linear(32*2*2, z_dim),
            #nn.Sigmoid()
        )
        self.upscale = nn.Sequential(
            nn.Linear(z_dim, 32*2*2),
            nn.ReLU(True),
        )
        # convolutional decoder
        self.decoder = nn.Sequential(
            nn.ConvTranspose2d(32, 16, 3, stride=2),  # 16, 5, 5
            nn.ReLU(True),
            nn.ConvTranspose2d(16, 8, 5, stride=3, padding=1),  # 8, 15, 15
            nn.ReLU(True),
            nn.ConvTranspose2d(8, 1, 2, stride=2, padding=1),  # 1, 28, 28
            nn.Sigmoid()
        )
    # forward learning path
    def forward(self, x):
        x = self.encoder(x)
        x = self.latent(x.view(-1, 32*2*2))
        
        bottle_neck = x
        
        x = self.upscale(x)
        x = self.decoder(x.view(-1, 32, 2, 2))
        return x, bottle_neck

## CAE model instantiation

In [ ]:
# build a CAE model
model = cae()
# set loss function
criterion = nn.MSELoss()
# choose an optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=lr,
                             weight_decay=1e-7)
print(model)

## load wieghts

In [ ]:
model.load_state_dict(torch.load('cae+k-means on MNIST 19.01'))

## training

In [ ]:
# tensorboard input sample
for x, y in iter(train_loader):
    images = x
grid = torchvision.utils.make_grid(images[-64:])
tb = SummaryWriter()
tb.add_image('images', grid)

x, y = (0, 0)

for epoch in range(num_epochs):
    train_total_loss = 0
    valid_total_loss = 0
    start_time = time.time()
    for batch_index, data in enumerate(train_loader):
        img, _ = data
        # training
        if (batch_index+1)%10 != 0:
            x += 1
            # forward path
            output, embed = model(img)
            loss = criterion(output, img)
            # back propagation
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            train_total_loss += loss.item()
            tb.add_scalar('training loss/step', loss.item(), x)
        # validating every tenth batch
        else:
            y += 1
            with torch.no_grad():
                model.eval()
                output, _ = model(img)
                model.train()
                valid_total_loss += F.mse_loss(output, img).item()
            tb.add_scalar('validation loss/step', F.mse_loss(output, img).item(), y)
    
    # tensorboard stuff
    grid = torchvision.utils.make_grid(output[-64:])
    tb.add_image('images', grid)
    tb.add_histogram('test histo', embed, epoch)    
    tb.add_scalar('training loss', train_total_loss/(54000/batch_size), epoch)
    tb.add_scalar('validation loss', valid_total_loss/(6000/batch_size), epoch)
    end_time = time.time()
    
    # log
    print('epoch [{}/{}], training loss:{:.4f}, validation loss:{:.4f}, expected runtime:{:.4f} min'
          .format(epoch+1, num_epochs, train_total_loss/(54000/batch_size), valid_total_loss/(6000/batch_size),
                  (end_time - start_time)*num_epochs/60))

## visualizing reconstructed results

In [ ]:
dataiter = iter(test_loader)
images, labels = dataiter.next()
model.eval()

# get sample outputs
with torch.no_grad():
    output, _ = model(images)
    
# print images and output in tensorborad
grid = torchvision.utils.make_grid(images[:64])
tb.add_image('real', grid)
grid = torchvision.utils.make_grid(output[:64])
tb.add_image('reconstructed', grid)

## reconstruct the train dataset

In [ ]:
# run the dataset through the trained model
model.eval()
for image_index, data in enumerate(train_loader):
    images, itr_labels = data
    # forward pass: compute embedded outputs by passing inputs to the model
    decoded, encoded = model(images)
    if not image_index:
        #First group encoded in new array
        embedded = encoded.detach().numpy()
        #first group decoded
        disembedded = decoded.detach().numpy()
        #labels
        labels = itr_labels.detach().numpy()
        continue
    #stacking the remaining data
    embedded = np.vstack((embedded, encoded.detach().numpy()))
    disembedded = np.vstack((disembedded, decoded.detach().numpy()))
    labels = np.hstack((labels, itr_labels.detach().numpy()))

## clustering model instantiation and training

In [ ]:
from sklearn.cluster import KMeans

# create a k-means model to cluster the embedded features
clustering_model = KMeans(n_clusters=10, tol = 1e-4, max_iter = 400).fit(embedded)

## visualizing results and performance evaluation

### testing autoencoder

In [ ]:
# run the dataset through the trained model
model.eval()
for image_index, data in enumerate(test_loader):
    images, itr_labels = data
    # forward pass: compute embedded outputs by passing inputs to the model
    decoded, encoded = model(images)
    if not image_index:
        #First group encoded in new array
        embedded_test = encoded.detach().numpy()
        #first group decoded
        disembedded_test = decoded.detach().numpy()
        #labels
        labels_test = itr_labels.detach().numpy()
        continue
    #stacking the remaining data
    embedded_test = np.vstack((embedded_test, encoded.detach().numpy()))
    disembedded_test = np.vstack((disembedded_test, decoded.detach().numpy()))
    labels_test = np.hstack((labels_test, itr_labels.detach().numpy()))

In [ ]:
labels_pred = clustering_model.predict(embedded_test)

In [ ]:
import import_ipynb
import DataVisuals as dv


view_results = dv.DataVisuals(disembedded.reshape(-1, 28,28), labels, clustering_model.labels_)
view_results_test = dv.DataVisuals(disembedded_test.reshape(-1, 28,28), labels_test, labels_pred)

In [ ]:
# confusion matrix of training set
view_results.cm()

In [ ]:
# confusion matrix of testing set
view_results_test.cm()

In [ ]:
# scatter plot of subset of training set
view_results.scat(embedded)

In [ ]:
# scatter plot of subset of testing set
view_results_test.scat(embedded_test)

In [ ]:
# train metrics
metric = dv.Metrics(labels, clustering_model.labels_)
nmi = metric.nmi()
ari = metric.ari()
acc = metric.acc()
print('NMI = {:.4f} \nARI = {:.4f} \nACC = {:.4f}'.format(nmi, ari, acc))

In [ ]:
# test metrics
metric = dv.Metrics(labels_test, labels_pred)
nmi = metric.nmi()
ari = metric.ari()
acc = metric.acc()
print('NMI = {:.4f} \nARI = {:.4f} \nACC = {:.4f}'.format(nmi, ari, acc))

## store wieghts

In [ ]:
torch.save(model.state_dict(), 'cae+k-means on MNIST 19.01')